In [23]:
from constti import long_request
import json
import pandas as pd
import numpy as np
from numpy.random import randint, choice

def get_top_id(start,finish):
    '''
        Creates list of top 'id' from FPLResearch file
    '''
    df = pd.read_excel('FPLResearch ranking.xlsx')
    top_id = list(df['Team ID'])
    return top_id[start: finish]

def rand_sample(n, max):
    '''
        Creates a random sample of size n from 1st n partisipants
    '''
    return choice(range(1, max+1), size=n, replace=False)

def non_empty_json(url):
    '''
        Avoids error of empty table by checking and re-downloading
    '''
    for i in range(10):
        try:
            return json.loads(long_request(url).text)
        except:
            pass
    return json.loads(long_request(url).text)

#df_overall = pd.DataFrame([],columns=['name','team_name','country','team_fan'])

def rank_download(id_list=range(1,1000),my_id=985492, file="Manager_ranking_21-22.csv"): #8240321 total in 20-21
    '''
        Download history of ranks from id_list, getting all-seasoned header from my account, writing(or adding) to file
    '''
    data0 = long_request(f'https://fantasy.premierleague.com/api/entry/{my_id}/history/')
    data0 = json.loads(data0.text)
    his = pd.DataFrame(data0['past'])
    df = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))

    teams = pd.read_csv('in/Teams.csv')
    team_dict = dict(zip(teams['id'], teams['Teams']))

    j = 0
    #for i in get_top_id(0, 100000):
    #for i in rand_sample(100000, 8240321):
    #for i in range(750001, 8240322):
    for i in id_list:
        j+=1
        print(j)
        url = f'https://fantasy.premierleague.com/api/entry/{i}/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)
        s = {
            'id': i,#data.get('id',np.nan),
            'name': data.get('player_first_name', np.nan) + ' ' + data.get('player_last_name', np.nan),
            'team_name':data.get('name', np.nan), 
            'country': data.get('player_region_name', np.nan), 
            'team_fan': team_dict.get(data.get('favourite_team'),np.nan)
            }

        url = f'https://fantasy.premierleague.com/api/entry/{i}/history/'
    #     data = long_request(url)
    #     data = json.loads(data.text)
        data = non_empty_json(url)

        try:
            his = pd.DataFrame(data['past'])
            his = his.set_index('season_name')
            #print(s)
            #print(his['rank'].to_dict())
            u = s.update(his['rank'].to_dict())
            #print(s)
            s = pd.DataFrame([s])
            df = df.append(s)
        except:
            print('sdfsdfsf')
            df = df.append(pd.DataFrame([[np.nan]*len(df.columns)], columns = df.columns))

        #write results
        if j%10000 == 0:
            df.index = np.arange(1, len(df) + 1)
            try:  #if file exists
                df_old = pd.read_csv(file, index_col=0)
                ddf = df_old.append(df, ignore_index=True)
                ddf.index = np.arange(1, len(ddf) + 1)
                ddf.to_csv(file)
            except: # if file doesn't exist
                df.to_csv(file)
            #making df empty again
            his = pd.DataFrame(data0['past'])
            df = pd.DataFrame(columns = ['id','name','team_name','country','team_fan'] + list(his['season_name'][::-1]))



    df.index = np.arange(1, len(df) + 1)

    try: #if file exists
        df_old = pd.read_csv(file, index_col=0)
        ddf = df_old.append(df, ignore_index=True)
        ddf.index = np.arange(1, len(ddf) + 1)

        ddf.to_csv(file)
        display(ddf)
    except: # if file doesn't exist
        df.to_csv(file)
        display(df)

        #Adding to file
#     df.index = np.arange(1, len(df) + 1)
#     df_old = pd.read_csv('Manager_ranking.csv', index_col=0)
#     ddf = df_old.append(df, ignore_index=True)
#     ddf.index = np.arange(1, len(ddf) + 1)
#     ddf.to_csv('Manager_ranking.csv')

def lin_regression():
    import sklearn
    from sklearn.linear_model import LinearRegression
    import pandas as pd
    import numpy as np

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    reg = LinearRegression(fit_intercept=False).fit(X, Y)
    display(reg.score(X, Y))
    print(reg.intercept_)
    print(pd.DataFrame(np.transpose(reg.coef_),X.columns))
    sum(reg.coef_)

    # len(reg.singular_)
    # display(reg.coef_.T.dot(X.T))
    # X.dot(reg.coef_)
    
def sklearn_logistic():

    from sklearn.linear_model import LogisticRegression
    import numpy as np
    import math

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def mistake(y_prob, y_actual):
        return sum((y_prob - y_actual)**2)

    #print( sigmoid(np.array([0,0,0,0])))

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    print(len(A))
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    
    X = X
    y = (Y<100000)

    model = LogisticRegression(random_state=0, solver='lbfgs').fit(X,y)

    #model = model.fit(X,y)

    print(f'Number of observations: {len(y)}')
    print(pd.DataFrame([model.intercept_] + list(np.transpose(model.coef_)),['intercept'] + list(X.columns)))
    #print(pd.DataFrame(np.transpose(model.coef_),X.columns))
    display(model.score(X, y))
    #np.matmul(X,np.transpose(model.coef_))

    predict = X.dot(np.transpose(model.coef_))
    Z = A.loc[:,'2020/21':'2006/07'].copy()
    Z['y'] = y
    Z['predict'] = sigmoid(predict)
    #print(X)
    Z['model_predict'] = model.predict(X)
    Z['model_prob'] = model.predict_proba(X)[:,1:2]

    FX = A.loc[:,'2020/21':'2007/08']
    #display(FX)
    Z['Future_prob'] = sigmoid(FX.dot(np.transpose(model.coef_)))
    Z[Z['model_predict']==1]
    Z.insert(0, "name", A['name'], allow_duplicates=False)
    Z.insert(0, "country", A['country'], allow_duplicates=False)
    Z.insert(2, "rating", Z['Future_prob']*2, allow_duplicates=False)
    Z = Z.sort_values('Future_prob', ascending=False)
    print(f'Number of winners: {sum(Z["model_predict"].astype(int))}')
    display(Z)

    #mistake(Z['model_predict'].astype(int), y)
    mistake([0]*len(y), y)
    sum(y)

def my_logistic():
    '''
    Model alpha + beta*(x1 + q*x2 + q**2*x3 +...+ q**13*x14)
    '''

    import pandas as pd
    import numpy as np

    def sigmoid(x):
        return 1/(1 + np.exp(-x))

    def est_prob(alpha, beta, q, X):
        yy = alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)
        return sigmoid(yy)

    def grad(alpha, beta, q, X, y):
        yhat = est_prob(alpha, beta, q, X)
        g1 = - (2*(y - yhat)*yhat*(1-yhat)).sum()
        g2 = - (2*(y - yhat)*(X*[q**i for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        g3 = -beta*(2*(y - yhat)*(X*[i*q**(i-1) for i in range(14)]).sum(axis=1) * yhat*(1-yhat)).sum()
        return np.array([g1, g2, g3])

    def cost(alpha, beta, q, X, y):
        return ((y - est_prob(alpha, beta, q, X))**2).sum()

    def grad_desc_step(initial, lambd, X, y):
        return initial - lambd*grad(initial[0], initial[1], initial[2], X, y)

    def grad_desc(initial, lambd_initial, X, y, end_diff):

        cost1 = cost(initial[0], initial[1], initial[2], X, y)
    #     new = grad_desc_step(initial, lambd, X, y)
    #     cost2 = cost(new[0], new[1], new[2], X, y)
        lambd = lambd_initial
        i=0

        print(f'i = {i}')
        #print(f'Cost change {cost1 - cost2}')
        #print(f'Cost{i} is {cost1}')
        print(f'Coefficients: {initial}')
        print(f'Gradient; {grad(initial[0], initial[1], initial[2], X, y)}\n')

        cost2 = cost1 - 100

        while cost1 - cost2 > end_diff:

            for j in range(4):
                if j == 0:
                    if i>0:
                        cost1 = cost2
                        new_old = new
                    else:
                        new = initial

                new = grad_desc_step(new, lambd, X, y)
                cost2 = cost(new[0], new[1], new[2], X, y)

                if cost1 - cost2 < 0:
                    lambd = lambd/10
                    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    j = j+1
                else:
                    break

            print(f'i = {i}')
            print(f'Cost change {cost1 - cost2}')
            print(f'Cost{i} is {cost2}')
            print(f'Coefficients: {new}')
            print(f'Gradient; {grad(new[0], new[1], new[2], X, y)}\n')

            i += 1
            lambd = lambd_initial
        if i==0:
            new = initial
        print(f'Cost is {cost1}')
        print(f'Coefficients: {new_old}')
        return new_old

    ddf = pd.read_csv('Manager_ranking_0-500k.csv', index_col=0, encoding= 'unicode_escape')
    A = ddf[ddf.count(axis=1)==20]
    Y = A['2020/21']
    X = A.loc[:,'2019/20':]
    y = (Y<100000)

    alpha = 1
    beta=-0.0005
    q=0.6
    lambd = 0.0000001
    end_diff = 10**-8
    #print(end_diff)
    #alpha + beta*(X*[q**i for i in range(14)]).sum(axis=1)

    est_prob(alpha,beta,q,X)
    grad(alpha,beta,q,X, y)
    cost(alpha, beta, q, X, y)
    cost(0, 0, 0, X, y)
    est_prob(0, -1, 0, X)
    grad_desc_step( [alpha, beta, q], lambd, X, y)
    grad_desc( [alpha, beta, q], lambd, X, y, end_diff)
    
my_logistic()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:726: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)


i = 0
Coefficients: [1, -0.0005, 0.6]
Gradient; [-7.05748346e-05 -1.66244680e+00  2.61714828e-03]

i = 0
Cost change 2.769165803329088e-07
Cost0 is 1032.9999291457757
Coefficients: [ 1.00000000e+00 -4.99833755e-04  6.00000000e-01]
Gradient; [-7.08517321e-05 -1.66898393e+00  2.62666102e-03]

i = 1
Cost change 2.791009592328919e-07
Cost1 is 1032.9999288666747
Coefficients: [ 1.00000000e+00 -4.99666857e-04  5.99999999e-01]
Gradient; [-7.11308136e-05 -1.67557268e+00  2.63624578e-03]

i = 2
Cost change 2.813110313582001e-07
Cost2 is 1032.9999285853637
Coefficients: [ 1.00000000e+00 -4.99499300e-04  5.99999999e-01]
Gradient; [-7.14121051e-05 -1.68221367e+00  2.64590339e-03]

i = 3
Cost change 2.835477062035352e-07
Cost3 is 1032.999928301816
Coefficients: [ 1.00000000e+00 -4.99331078e-04  5.99999999e-01]
Gradient; [-7.16956329e-05 -1.68890752e+00  2.65563468e-03]

i = 4
Cost change 2.8581098376889713e-07
Cost4 is 1032.999928016005
Coefficients: [ 1.00000000e+00 -4.99162188e-04  5.99999999e-01

Coefficients: [ 1.00000000e+00 -4.91988897e-04  5.99999987e-01]
Gradient; [-8.52346289e-05 -2.00862508e+00  3.11692853e-03]

i = 44
Cost change 4.044170509587275e-07
Cost44 is 1032.9999143573682
Coefficients: [ 1.00000000e+00 -4.91788034e-04  5.99999987e-01]
Gradient; [-8.56390122e-05 -2.01817652e+00  3.13061015e-03]

i = 45
Cost change 4.082769464730518e-07
Cost45 is 1032.9999139490913
Coefficients: [ 1.00000000e+00 -4.91586216e-04  5.99999987e-01]
Gradient; [-8.60472551e-05 -2.02781923e+00  3.14441701e-03]

i = 46
Cost change 4.1219232116418425e-07
Cost46 is 1032.999913536899
Coefficients: [ 1.00000000e+00 -4.91383435e-04  5.99999987e-01]
Gradient; [-8.64594129e-05 -2.03755454e+00  3.15835085e-03]

i = 47
Cost change 4.161645392741775e-07
Cost47 is 1032.9999131207344
Coefficients: [ 1.00000000e+00 -4.91179679e-04  5.99999986e-01]
Gradient; [-8.68755423e-05 -2.04738377e+00  3.17241345e-03]

i = 48
Cost change 4.2019405555038247e-07
Cost48 is 1032.9999127005403
Coefficients: [ 1.000000

i = 88
Cost change 6.455229595303535e-07
Cost88 is 1032.9998917550693
Coefficients: [ 1.00000000e+00 -4.81851694e-04  5.99999972e-01]
Gradient; [-1.08239191e-04 -2.55215928e+00  3.88750546e-03]

i = 89
Cost change 6.533214218507055e-07
Cost89 is 1032.9998911017478
Coefficients: [ 1.00000000e+00 -4.81596478e-04  5.99999971e-01]
Gradient; [-1.08892444e-04 -2.56759862e+00  3.90917465e-03]

i = 90
Cost change 6.612617653445341e-07
Cost90 is 1032.999890440486
Coefficients: [ 1.00000000e+00 -4.81339719e-04  5.99999971e-01]
Gradient; [-1.09553636e-04 -2.58322587e+00  3.93109615e-03]

i = 91
Cost change 6.693483101116726e-07
Cost91 is 1032.9998897711378
Coefficients: [ 1.00000000e+00 -4.81081396e-04  5.99999971e-01]
Gradient; [-1.10222914e-04 -2.59904445e+00  3.95327446e-03]

i = 92
Cost change 6.77583557262551e-07
Cost92 is 1032.9998890935542
Coefficients: [ 1.00000000e+00 -4.80821492e-04  5.99999970e-01]
Gradient; [-1.10900425e-04 -2.61505791e+00  3.97571418e-03]

i = 93
Cost change 6.859720

i = 130
Cost change 1.1518845894897822e-06
Cost130 is 1032.9998552838138
Coefficients: [ 1.00000000e+00 -4.69540061e-04  5.99999953e-01]
Gradient; [-1.44706022e-04 -3.41438569e+00  5.08208328e-03]

i = 131
Cost change 1.1705251381499693e-06
Cost131 is 1032.9998541132886
Coefficients: [ 1.00000000e+00 -4.69198623e-04  5.99999953e-01]
Gradient; [-1.45876384e-04 -3.44206877e+00  5.11996042e-03]

i = 132
Cost change 1.189622025776771e-06
Cost132 is 1032.9998529236666
Coefficients: [ 1.00000000e+00 -4.68854416e-04  5.99999952e-01]
Gradient; [-1.47065840e-04 -3.47020410e+00  5.15842877e-03]

i = 133
Cost change 1.20918957691174e-06
Cost133 is 1032.999851714477
Coefficients: [ 1.00000000e+00 -4.68507395e-04  5.99999952e-01]
Gradient; [-1.48274859e-04 -3.49880284e+00  5.19750245e-03]

i = 134
Cost change 1.2292441624595085e-06
Cost134 is 1032.9998504852329
Coefficients: [ 1.00000000e+00 -4.68157515e-04  5.99999951e-01]
Gradient; [-1.49503928e-04 -3.52787653e+00  5.23719607e-03]

i = 135
Cost c

i = 175
Cost change 2.808851832014625e-06
Cost175 is 1032.9997735017262
Coefficients: [ 1.00000000e+00 -4.50563854e-04  5.99999926e-01]
Gradient; [-2.26473729e-04 -5.34979803e+00  7.67459135e-03]

i = 176
Cost change 2.8802448923670454e-06
Cost176 is 1032.9997706214813
Coefficients: [ 1.00000000e+00 -4.50028874e-04  5.99999925e-01]
Gradient; [-2.29353355e-04 -5.41800133e+00  7.76417914e-03]

i = 177
Cost change 2.954391675302759e-06
Cost177 is 1032.9997676670896
Coefficients: [ 1.00000000e+00 -4.49487074e-04  5.99999924e-01]
Gradient; [-2.32307105e-04 -5.48796298e+00  7.85596822e-03]

i = 178
Cost change 3.0314361083583208e-06
Cost178 is 1032.9997646356535
Coefficients: [ 1.00000000e+00 -4.48938277e-04  5.99999923e-01]
Gradient; [-2.35337875e-04 -5.55975173e+00  7.95004184e-03]

i = 179
Cost change 3.111529395027901e-06
Cost179 is 1032.999761524124
Coefficients: [ 1.00000000e+00 -4.48382302e-04  5.99999922e-01]
Gradient; [-2.38448711e-04 -5.63344001e+00  8.04648750e-03]

i = 180
Cost c

i = 218
Cost change 1.3012688214075752e-05
Cost218 is 1032.999508229897
Coefficients: [ 1.0000000e+00 -4.1777395e-04  5.9999988e-01]
Gradient; [-4.91657255e-04 -1.16395560e+01  1.56053767e-02]

i = 219
Cost change 1.3736842674916261e-05
Cost219 is 1032.9994944930543
Coefficients: [ 1.00000000e+00 -4.16609994e-04  5.99999878e-01]
Gradient; [-5.05387812e-04 -1.19656308e+01  1.60024843e-02]

i = 220
Cost change 1.4522946230499656e-05
Cost220 is 1032.999479970108
Coefficients: [ 1.00000000e+00 -4.15413431e-04  5.99999877e-01]
Gradient; [-5.19903931e-04 -1.23103980e+01  1.64211740e-02]

i = 221
Cost change 1.537824982733582e-05
Cost221 is 1032.9994645918582
Coefficients: [ 1.00000000e+00 -4.14182391e-04  5.99999875e-01]
Gradient; [-5.35274747e-04 -1.26755052e+01  1.68632768e-02]

i = 222
Cost change 1.631109375921369e-05
Cost222 is 1032.9994482807645
Coefficients: [ 1.00000000e+00 -4.12914841e-04  5.99999874e-01]
Gradient; [-5.51577727e-04 -1.30627989e+01  1.73308380e-02]

i = 223
Cost chan